In [ ]:
# export
from fastai_local.core import *
from fastai_local.test import *
from fastai_local.imports import *
import nbformat

In [ ]:
# default_exp notebook.export

# Converting notebooks to modules

> The functions that transform the dev notebooks in the fastai library

## Reading a notebook

### What's a notebook?

A jupyter notebook is a json file behind the scenes. We can just read it with the json module, which will return a nested dictionary of dictionaries/lists of dictionaries, but there are some small differences between reading the json and using the tools from `nbformat` so we'll use this one.

In [ ]:
#export
def read_nb(fname):
    "Read the notebook in `fname`."
    with open(Path(fname),'r') as f: return nbformat.reads(f.read(), as_version=4)

`fname` can be a string or a pathlib object.

In [ ]:
test_nb = read_nb('99_export.ipynb')

The root has four keys: `cells` contains the cells of the notebook, `metadata` some stuff around the version of python used to execute the notebook, `nbformat` and `nbformat_minor` the version of nbformat. 

In [ ]:
test_nb.keys()

dict_keys(['cells', 'metadata', 'nbformat', 'nbformat_minor'])

In [ ]:
test_nb['metadata']

{'kernelspec': {'display_name': 'Python 3',
  'language': 'python',
  'name': 'python3'},
 'language_info': {'codemirror_mode': {'name': 'ipython', 'version': 3},
  'file_extension': '.py',
  'mimetype': 'text/x-python',
  'name': 'python',
  'nbconvert_exporter': 'python',
  'pygments_lexer': 'ipython3',
  'version': '3.7.1'}}

In [ ]:
f"{test_nb['nbformat']}.{test_nb['nbformat_minor']}"

'4.2'

The cells key then contains a list of cells. Each one is a new dictionary that contains entries like the type (code or markdown), the source (what is written in the cell) and the output (for code cells).

In [ ]:
test_nb['cells'][0]

{'cell_type': 'code',
 'execution_count': 1,
 'metadata': {'hide_input': False},
 'outputs': [],
 'source': '# export\nfrom fastai_local.core import *\nfrom fastai_local.test import *\nfrom fastai_local.imports import *\nimport nbformat'}

### Finding patterns

In [ ]:
# export
def check_re(cell, pat):
    if cell['cell_type'] != 'code': return False
    return re.match(pat, cell['source'], re.IGNORECASE | re.MULTILINE)

In [ ]:
cell = test_nb['cells'][0].copy()
assert check_re(cell, '# export') is not None
test_is(check_re(cell, '# bla'), None)

In [ ]:
# export
def is_export(cell, default):
    "Check if `cell` is to be exported and returns the name of the module."
    if check_re(cell, r'^\s*#\s*exports?\s*$'): 
        if default is None: print(f"This cell doesn't have an export destination and was ignored:\n{cell['source'][1]}")
        return default
    tst = check_re(cell, r'^\s*#\s*exports?\s*(\S+)\s*$')
    return os.path.sep.join(tst.groups()[0].split('.')) if tst else None 

The cells to export are marked with an `#export` or `#exports` code, potentially with a module name where we want it exported. The default is given in a cell of the form `#default_exp bla` inside the notebook (usually at the top), though in this function, it needs the be passed (the final script will read the whole notebook to find it).

In [ ]:
cell = test_nb['cells'][0].copy()
test_eq(is_export(cell, 'export'), 'export')
cell['source'] = "# exports" 
test_eq(is_export(cell, 'export'), 'export')
cell['source'] = "# export mod" 
test_eq(is_export(cell, 'export'), 'mod')
cell['source'] = "# export mod.file" 
test_eq(is_export(cell, 'export'), 'mod/file')
cell['source'] = "# expt mod.file"
assert is_export(cell, 'export') is None

In [ ]:
# export
def find_default_export(cells):
    "Find in `cells` the default export module."
    for cell in cells:
        tst = check_re(cell, r'^\s*#\s*default_exp\s*(\S*)\s*$')
        if tst: return tst.groups()[0]

Stops at the first cell containing a `#default_exp` code and return the value behind. Returns `None` if there are no cell with that code.

In [ ]:
test_eq(find_default_export(test_nb['cells']), 'notebook.export')
assert find_default_export(test_nb['cells'][2:]) is None

### Exporting notebooks

We're now ready to export notebooks!

In [ ]:
# export
def _create_mod_file(fname, nb_path):
    "Create a module file for `fname`."
    fname.parent.mkdir(parents=True, exist_ok=True)
    with open(fname, 'w') as f: 
        f.write(f"#AUTOGENERATED! DO NOT EDIT! File to edit: dev/{nb_path.name} (unless otherwise specified).")
        f.write('\n\n__all__ = []')

In [ ]:
# export
def export_names(code, func_only=False):
    "Find the names of the objects, functions or classes defined in `code` that are exported."
    names = re.findall(r'^(?:def|class)\s+([^\(\s]*)\s*\(', code, re.MULTILINE)
    if not func_only: names += re.findall(r'^([^\(\s]*)\s*=', code, re.MULTILINE)
    return [n for n in names if not n.startswith('_') and not '.' in n]

This function only picks the zero-indented objects, functions or classes (we don't want the class methods for instance) and excludes private names (that begin with `_`). It only returns func and class names when `func_only=True`.

In [ ]:
test_eq(export_names("def my_func(x)():\n  pass\nclass MyClass():"), ["my_func", "MyClass"])
#Indented funcs are ignored (funcs inside a class)
test_eq(export_names("  def my_func(x)():\n  pass\nclass MyClass():"), ["MyClass"])
#Private funcs are ignored
test_eq(export_names("def _my_func():\n  pass\nclass MyClass():"), ["MyClass"])
#trailing spaces
test_eq(export_names("def my_func ():\n  pass\nclass MyClass():"), ["my_func", "MyClass"])
#object and funcs
test_eq(export_names("def my_func ():\n  pass\ndefault_bla = []:"), ["my_func", "default_bla"])
test_eq(export_names("def my_func ():\n  pass\ndefault_bla = []:", func_only=True), ["my_func"])
#Private objects are ignored
test_eq(export_names("def my_func ():\n  pass\n_default_bla = []:"), ["my_func"])
#Objects with dots are ignored
test_eq(export_names("def my_func ():\n  pass\ndefault.bla = []:"), ["my_func"])

In [ ]:
#export
def _add2add(fname, names, line_width=120):
    if len(names) == 0: return
    with open(fname, 'r') as f: text = f.read()
    tw = TextWrapper(width=120, initial_indent='', subsequent_indent=' '*11, break_long_words=False)
    re_all = re.search(r'__all__\s*=\s*\[([^\]]*)\]', text)
    start,end = re_all.start(),re_all.end()
    text_all = tw.wrap(f"{text[start:end-1]}{'' if text[end-2]=='[' else ', '}{', '.join(names)}]")
    with open(fname, 'w') as f: f.write(text[:start] + '\n'.join(text_all) + text[end:])

In [ ]:
fname = 'test_add.txt'
with open(fname, 'w') as f: f.write("Bla\n__all__ = [my_file, MyClas]\nBli")
_add2add(fname, ['new_function'])
with open(fname, 'r') as f: 
    test_eq(f.read(), "Bla\n__all__ = [my_file, MyClas, new_function]\nBli")
_add2add(fname, [f'new_function{i}' for i in range(10)])
with open(fname, 'r') as f: 
    test_eq(f.read(), """Bla
__all__ = [my_file, MyClas, new_function, new_function0, new_function1, new_function2, new_function3, new_function4,
           new_function5, new_function6, new_function7, new_function8, new_function9]
Bli""")
os.remove(fname)

In [ ]:
# export
def _relative_import(name, fname):
    mods = name.split('.')
    splits = str(fname).split(os.path.sep)
    if mods[0] not in splits: return name
    splits = splits[splits.index(mods[0]):]
    while splits[0] == mods[0]: splits,mods = splits[1:],mods[1:]
    return '.' * (len(splits)-len(mods)+1) + '.'.join(mods)

In [ ]:
test_eq(_relative_import('fastai_local.core', Path('fastai_local')/'data.py'), '.core')
test_eq(_relative_import('fastai_local.core', Path('fastai_local')/'vision'/'data.py'), '..core')
test_eq(_relative_import('fastai_local.vision.transform', Path('fastai_local')/'vision'/'data.py'), '.transform')

In [ ]:
pat = re.compile(r'from (fastai_local.\S*) import (\S*)$')
m = re.match(pat, "from fastai_local.core import *\n")
m.groups()

('fastai_local.core', '*')

In [ ]:
# export
def _deal_import(code_lines, fname):
    pat = re.compile(r'from (fastai_local.\S*) import (\S*)$')
    lines = []
    for line in code_lines:
        match = re.match(pat, line)
        if match: lines.append(f"from {_relative_import(match.groups()[0], fname)} import {match.groups()[1]}")
        else: lines.append(line)
    return lines                    

In [ ]:
#hide
#Tricking jupyter notebook to have a __file__ attribute.
__file__ = Path('fastai_local').absolute()/'notebook'/'export.py'

In [ ]:
#export
def _get_index():
    if not (Path(__file__).parent/'index.txt').exists(): return {}
    return json.load(open(Path(__file__).parent/'index.txt', 'r'))

def _save_index(index): json.dump(index, open(Path(__file__).parent/'index.txt', 'w'))

In [ ]:
#hide
ind,ind_bak = Path(__file__).parent/'index.txt',Path(__file__).parent/'index.bak'
if ind.exists(): shutil.move(ind, ind_bak)
test_eq(_get_index(), {})
_save_index({'foo':'bar'})
test_eq(_get_index(), {'foo':'bar'})
if ind_bak.exists(): shutil.move(ind_bak, ind)

In [ ]:
#export 
def _notebook2script(fname):
    "Finds cells starting with `#export` and puts them into a new module"
    fname = Path(fname)
    nb = read_nb(fname)
    default = find_default_export(nb['cells'])
    if default is not None: 
        default = os.path.sep.join(default.split('.'))
        _create_mod_file(Path.cwd()/'fastai_local'/f'{default}.py', fname)
    index = _get_index()
    exports = [is_export(c, default) for c in nb['cells']]
    cells = [(c,e) for (c,e) in zip(nb['cells'],exports) if e is not None]
    for (c,e) in cells:
        fname_out = Path.cwd()/'fastai_local'/f'{e}.py'
        orig = '' if e==default else f'#Comes from {fname.name}.\n'
        code = '\n\n' + orig + '\n'.join(_deal_import(c['source'].split('\n')[1:], fname_out))
        # remove trailing spaces
        names = export_names(code)
        _add2add(fname_out, [f"'{f}'" for f in names])
        index.update({f: fname.name for f in names})
        code = re.sub(r' +$', '', code, flags=re.MULTILINE)
        with open(fname_out, 'a') as f: f.write(code)
    _save_index(index)
    print(f"Converted {fname}.")

In [ ]:
#export 
def _get_sorted_files(all_fs: Union[bool,str], up_to=None):
    "Return the list of files corresponding to `g` in the current dir."
    if (all_fs==True): ret = glob.glob('*.ipynb') # Checks both that is bool type and that is True
    else: ret = glob.glob(all_fs) if isinstance(g,str) else []
    if len(ret)==0: print('WARNING: No files found')
    if up_to is not None: ret = [f for f in ret if str(f)<=str(up_to)]
    return sorted(ret)

In [ ]:
#export 
def notebook2script(fname=None, all_fs=None, up_to=None):
    "Convert `fname` or all the notebook satisfying `all_fs`."
    # initial checks
    assert fname or all_fs
    if (all_fs is None) and (up_to is not None): all_fs=True # Enable allFiles if upTo is present
    fnames = _get_sorted_files(all_fs, up_to=up_to) if all_fs else [fname]
    [_notebook2script(f) for f in fnames]

Finds cells starting with `#export` and puts them into the appropriate module.
* `fname`: the filename of one notebook to convert
* `all_fs`: `True` if you want to convert all notebook files in the folder or a glob expression
* `up_to`: converts all notebooks respecting the previous arg up to a certain number

Examples of use in console:
```
notebook2script                                 # Parse all files
notebook2script --fname 00_export.ipynb         # Parse 00_export.ipynb
notebook2script --all_fs=nb*                    # Parse all files starting with nb*
notebook2script --up_to=10                      # Parse all files with (name<='10')
notebook2script --all_fs=*_*.ipynb --up_to=10   # Parse all files with an '_' and (name<='10')
```

### Finding the way back to notebooks

In [ ]:
# export
def source_nb(func):
    "Return the name of the notebook where `func` was defined"
    index = _get_index()
    return index.get(func.__name__, "Not found")

In [ ]:
from fastai_local.data.pipeline import Transform
test_eq(source_nb(test_fail),'00_test.ipynb')
test_eq(source_nb(Transform),'02_data_pipeline.ipynb')
#add_docs is in the core module but defined in 02
test_eq(source_nb(add_docs), '02_data_pipeline.ipynb')

In [ ]:
#hide
notebook2script(all_fs=True)

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "/home/ubuntu/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py", line 3267, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-72-172b19f8ef39>", line 2, in <module>
    notebook2script(all_fs=True)
  File "<ipython-input-67-bb67b32d78c9>", line 8, in notebook2script
    [_notebook2script(f) for f in fnames]
  File "<ipython-input-67-bb67b32d78c9>", line 8, in <listcomp>
    [_notebook2script(f) for f in fnames]
  File "<ipython-input-65-0d434165917a>", line 10, in _notebook2script
    index = _get_index()
  File "<ipython-input-63-454d00475b74>", line 4, in _get_index
    return json.load(open(Path(__file__).parent/'index.txt', 'r'))
  File "/home/ubuntu/anaconda3/lib/python3.7/json/__init__.py", line 296, in load
    parse_constant=parse_constant, object_pairs_hook=object_pairs_hook, **kw)
  File "/home/ubuntu/anaconda3/lib/python3.7/json/__init__.py", line 348, in loads
    return _

JSONDecodeError: Expecting value: line 1 column 1 (char 0)